## Feature Engineering

In the following cells, we will engineer the variables of the House Price Dataset so that we tackle:

1. Missing values
2. Temporal variables
3. Non-Gaussian distributed variables
4. Categorical variables: remove rare labels
5. Categorical variables: convert strings to numbers
5. Put the variables in a similar scale

In [88]:
# data wrangling
import pandas as pd
import numpy as np

# plotting
import matplotlib.pyplot as plt

# yeo-johnson transformation
import scipy.stats as stats

# feature scaling and data mining
from sklearn.model_selection import train_test_split


# to save the trained scaler class
import joblib

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

In [89]:
data = pd.read_csv('train.csv')

print(data.shape)

data.head(1)

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500


In [90]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['Id', 'SalePrice'], axis=1),
    data['SalePrice'],
    test_size = 0.1,
    random_state=0
)

X_train.shape, X_test.shape

((1314, 79), (146, 79))

### Target

In [91]:
y_train = np.log(y_train)
y_test = np.log(y_test)

## Missing Values

### Categorical variables

Variables with lots of missing values will be replaced with "missing". Variables with fewer missing values ibservation with the most frequent

In [92]:
cat_vars = [var for var in data.columns if data[var].dtype == 'O']

cat_vars = cat_vars + ['MSSubClass']


X_train[cat_vars] = X_train[cat_vars].astype('O')
X_test[cat_vars] = X_test[cat_vars].astype('O')


len(cat_vars)

44

In [93]:
cat_vars_with_na = [
    var for var in cat_vars
    if X_train[var].isnull().sum() > 0
]


X_train[cat_vars_with_na].isnull().mean().sort_values(
    ascending=False
)

PoolQC          0.995434
MiscFeature     0.961187
Alley           0.938356
Fence           0.814307
FireplaceQu     0.472603
GarageType      0.056317
GarageFinish    0.056317
GarageQual      0.056317
GarageCond      0.056317
BsmtExposure    0.025114
BsmtFinType2    0.025114
BsmtQual        0.024353
BsmtCond        0.024353
BsmtFinType1    0.024353
MasVnrType      0.004566
Electrical      0.000761
dtype: float64

In [94]:
# for columns with missing value greater than 10%
# replace with 'missing'
# While for those less than 10% replace with the first mode


with_string_missing = [
    var for var in cat_vars_with_na if X_train[var].isnull().mean()
    > 0.1 
]


with_mode_missing = [
    var for var in cat_vars_with_na if X_train[var].isnull().mean()
    < 0.1
]

In [95]:
with_string_missing

['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

In [96]:
X_train[with_string_missing] = X_train[with_string_missing].fillna('Missing')
X_test[with_string_missing] = X_test[with_string_missing].fillna('missing')

In [97]:
with_mode_missing

['MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [98]:
missing_mode = X_train[with_mode_missing].mode()

In [99]:
missing_mode

,MasVnrType,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,GarageType,GarageFinish,GarageQual,GarageCond
0,None,TA,TA,No,Unf,Unf,SBrkr,Attchd,Unf,TA,TA


In [100]:
for var in with_mode_missing:

    mode = X_train[var].mode()[0]

    print('{0} mode is ==> {1}'.format(var, mode))

    X_train[var].fillna(mode, inplace=True)
    X_test[var].fillna(mode, inplace=True)



MasVnrType mode is ==> None
BsmtQual mode is ==> TA
BsmtCond mode is ==> TA
BsmtExposure mode is ==> No
BsmtFinType1 mode is ==> Unf
BsmtFinType2 mode is ==> Unf
Electrical mode is ==> SBrkr
GarageType mode is ==> Attchd
GarageFinish mode is ==> Unf
GarageQual mode is ==> TA
GarageCond mode is ==> TA


In [101]:
# comfirm there are no more missing values

X_train[cat_vars_with_na].isnull().sum()

Alley           0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
Electrical      0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
dtype: int64

### Numerical variables

To engineer missing values for numerical variables, we will:


1. Add a binary indicator variable column
2. Replace the missing values in the original variable with mean

In [102]:
num_vars = [
    var for var in X_train.columns if var not in cat_vars and var != 'SalePrice'
]

# number of numerical variables
len(num_vars)


35

In [103]:
# list of numerical variables with missing values

vars_with_na = [
    var for var in num_vars if X_train[var].isnull().sum() > 0
]


X_train[vars_with_na].isnull().mean()



LotFrontage    0.177321
MasVnrArea     0.004566
GarageYrBlt    0.056317
dtype: float64

In [104]:
# replace missing values with mean
# create binary indicator columns

for var in vars_with_na:

    mean = X_train[var].mean()

    print(var, mean)

    X_train[var + '_na'] = np.where(X_train[var].isnull(), 1, 0)
    X_test[var + '_na'] = np.where(X_test[var].isnull(), 1, 0)


    X_train[var].fillna(mean, inplace=True)
    X_test[var].fillna(mean, inplace=True)

X_train[vars_with_na].isnull().sum()
    

LotFrontage 69.87974098057354
MasVnrArea 103.7974006116208
GarageYrBlt 1978.2959677419356


LotFrontage    0
MasVnrArea     0
GarageYrBlt    0
dtype: int64

In [105]:
# double check there are no non values in the test data
X_test[vars_with_na].isnull().sum()

LotFrontage    0
MasVnrArea     0
GarageYrBlt    0
dtype: int64

In [106]:
[var for var in vars_with_na if X_test[var].isnull().sum() > 0]

[]

In [107]:
# Have a look at the binary indicator column

X_train[['LotFrontage_na', 'MasVnrArea_na', 'GarageYrBlt_na']].head(3)

,LotFrontage_na,MasVnrArea_na,GarageYrBlt_na
930,0,0,0
656,0,0,0
45,0,0,0


### Temporal (year) Variable

In [108]:
def elapsed_years(df, var):

    df[var] = df['YrSold'] - df[var]
    return df

In [109]:
for var in ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']:

    X_train = elapsed_years(X_train, var)
    X_test = elapsed_years(X_test, var)

In [110]:
X_train.drop(['YrSold'], axis = 1, inplace=True)
X_test.drop(['YrSold'], axis = 1, inplace=True)

### Logarithmic Transformation

We will transform with the logarightm the positive numerical variables in order to get a more Gaussian-like distribution.

In [111]:
for var in ["LotFrontage", "1stFlrSF", "GrLivArea"]:

    X_train[var] = np.log(X_train[var])
    X_test[var] = np.log(X_test[var])  

In [112]:
# confirm that there are no null values in the extracted tweets

[var for var in ["LotFrontage", "1stFlrSF", "GrLivArea"] if X_test[var].isnull().sum() > 0]

[]

In [113]:
[var for var in ["LotFrontage", "1stFlrSF", "GrLivArea"] if X_train[var].isnull().sum() > 0]

[]

### Yeo-transformation

In [114]:
X_train['LotArea'], param = stats.yeojohnson(X_train['LotArea'])

X_test['LotArea'] = stats.yeojohnson(X_test['LotArea'], lmbda=param)


print(param)

-12.55283001172003


/home/arnold/Desktop/MlOp/env/lib/python3.8/site-packages/scipy/stats/morestats.py:1557: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(trans.var(axis=0))
/home/arnold/Desktop/MlOp/env/lib/python3.8/site-packages/scipy/optimize/optimize.py:2621: RuntimeWarning: invalid value encountered in double_scalars
  w = xb - ((xb - xc) * tmp2 - (xb - xa) * tmp1) / denom
/home/arnold/Desktop/MlOp/env/lib/python3.8/site-packages/scipy/optimize/optimize.py:2214: RuntimeWarning: invalid value encountered in double_scalars
  tmp1 = (x - w) * (fx - fv)
/home/arnold/Desktop/MlOp/env/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


In [115]:
[var for var in X_train.columns if X_train[var].isnull().sum() > 0]

[]

In [116]:
[var for var in X_train.columns if X_test[var].isnull().sum() > 0]

[]